Consider `M` as the transition probability matrix, `p` as the probability of jumping instead of using `M`, `N` being total number of pages and `q` being a vector of 1s since the jump is uniformly distributed. PageRank can be written as 

```
PR = (1-p)*M*PR + p*(1/N)*q
```



As `p` increases, the chance for jumping increases as well and the variance of values in the PageRank vector will decrease as a result because the jumping distribution is uniform. `p` increases until 1, where every move is a jump so the PageRank ends up being uniform.

Using the graph in Tutorial 4 Q2 

![graphimg](graph.png)

below is a demonstration of the change in `p`

In [2]:
import numpy as np
np.set_printoptions(precision=3)

ϵ = 1e-6

def create_transition_matrix(adj_matrix):
    out_degrees = np.sum(adj_matrix, axis=0)
    return adj_matrix / out_degrees

def get_page_rank(matrix: np.ndarray, p: float, q: np.ndarray=None, max_iterations:int=1000):
    N = matrix.shape[0]
    if q is None:
        q = np.ones((N, N))
    transition_matrix = create_transition_matrix(matrix)
    teleportation_matrix = p / N * q

    google_matrix = (1-p) * transition_matrix + teleportation_matrix
    
    page_rank = np.ones(N) / N
    for _ in range(max_iterations):
        new_page_rank = google_matrix @ page_rank
        if np.linalg.norm(new_page_rank - page_rank) < ϵ:
            break
        page_rank = new_page_rank

    return page_rank

adj_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0],
], dtype=float)

for p in np.arange(0, 1.2, 0.2):
    print(f"[p={p:.1f}]")
    page_rank = get_page_rank(adj_matrix, p)
    var = np.var(page_rank)
    print(f"PageRank: {page_rank}")
    print(f"Variance: {var:.4f}\n")

[p=0.0]
PageRank: [0.139 0.175 0.211 0.141 0.1   0.026 0.052 0.156]
Variance: 0.0034

[p=0.2]
PageRank: [0.126 0.154 0.198 0.153 0.107 0.051 0.064 0.147]
Variance: 0.0021

[p=0.4]
PageRank: [0.12  0.14  0.18  0.157 0.114 0.073 0.078 0.138]
Variance: 0.0012

[p=0.6]
PageRank: [0.119 0.131 0.16  0.154 0.12  0.093 0.092 0.131]
Variance: 0.0005

[p=0.8]
PageRank: [0.121 0.126 0.141 0.143 0.124 0.111 0.108 0.126]
Variance: 0.0001

[p=1.0]
PageRank: [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
Variance: 0.0000



Now if `q`, or the teleportation matrix, is variable, it would be defined to fit the user's need. Having `q` be variable adds bias to the nodes and those with a higher probability will have a higher PageRank value as a result. 

For example, if a user does a search with tags (like searching `Delicious fruits` with tags like `Red`, `Round`, `Cheap`, `Seasonal`, etc), the q matrix can have the pages with more tags matched have a higher probability of being jumped to.

The example follow the same graph setup as above. For the q vector, it will be based off how many tags the page has matched with the user's search, basically a normalised vector of the count of tags matched. p will be a constant 0.15.

The numbers of tags matched for page 1 will increment to show the changes in how teleporting bias can affect the PageRank.

In [38]:
import numpy as np

def normalize_columns(matrix):
    column_sums = matrix.sum(axis=0)
    return matrix / column_sums

def create_transition_matrix(matrix, p: float, q: np.ndarray = None):
    if q is None:
        q = np.full((8, 8), 1/8)
        
    norm_adj_matrix = normalize_columns(matrix)
    teleportation_contribution = p * q
    transition_matrix = (1-p) * norm_adj_matrix + teleportation_contribution
    return transition_matrix

def get_page_rank(matrix, num_iterations=100):
    N = matrix.shape[0]
    rank_vector = np.ones(N) / N
    for _ in range(num_iterations):
        new_rank_vector = matrix.dot(rank_vector)
        if np.linalg.norm(new_rank_vector - rank_vector) < ϵ:
            break
        rank_vector = new_rank_vector
    return rank_vector

adj_matrix = np.array([
    [0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0],
])

p = 0.15

tag_count = np.array([0, 1, 3, 4, 2, 1, 1, 2], dtype=float)

for i in range(0, 31, 6):
    tag_count[0] = i

    q = np.array(list(map(
        lambda x: [x for _ in adj_matrix],
        tag_count / sum(tag_count)
    )), dtype=float)

    transition_matrix = create_transition_matrix(adj_matrix, p, q)
    page_rank = get_page_rank(transition_matrix)

    print(f"[{tag_count[0]} Tags Matched]")
    print(f"Tag Count: {tag_count}")
    print(f"q: {tag_count / sum(tag_count)}")
    print(f"PageRank: {page_rank}\n")


[0.0 Tags Matched]
Tag Count: [0. 1. 3. 4. 2. 1. 1. 2.]
q: [0.    0.071 0.214 0.286 0.143 0.071 0.071 0.143]
PageRank: [0.109 0.151 0.222 0.169 0.1   0.034 0.056 0.158]

[6.0 Tags Matched]
Tag Count: [6. 1. 3. 4. 2. 1. 1. 2.]
q: [0.3  0.05 0.15 0.2  0.1  0.05 0.05 0.1 ]
PageRank: [0.156 0.164 0.213 0.146 0.097 0.028 0.049 0.147]

[12.0 Tags Matched]
Tag Count: [12.  1.  3.  4.  2.  1.  1.  2.]
q: [0.462 0.038 0.115 0.154 0.077 0.038 0.038 0.077]
PageRank: [0.182 0.171 0.208 0.133 0.095 0.025 0.046 0.14 ]

[18.0 Tags Matched]
Tag Count: [18.  1.  3.  4.  2.  1.  1.  2.]
q: [0.562 0.031 0.094 0.125 0.062 0.031 0.031 0.062]
PageRank: [0.198 0.176 0.205 0.125 0.094 0.023 0.043 0.136]

[24.0 Tags Matched]
Tag Count: [24.  1.  3.  4.  2.  1.  1.  2.]
q: [0.632 0.026 0.079 0.105 0.053 0.026 0.026 0.053]
PageRank: [0.209 0.179 0.203 0.12  0.093 0.022 0.042 0.134]

[30.0 Tags Matched]
Tag Count: [30.  1.  3.  4.  2.  1.  1.  2.]
q: [0.682 0.023 0.068 0.091 0.045 0.023 0.023 0.045]
PageRank: [0.